In [37]:
path = "cmake-build-debug/Build/bin/"

max_iters = 1000000

dimensions = [5,10,20]

cmds = [{ 
            "name": "gauss_newton",
            "cmd" : "GaussNewton {function_index} {x} {y} {max_iter} {filename}"
        },
        {
            "name": "lbfgs_m20",
            "cmd" : "LBFGS {function_index} {x} {y} {m} {max_iter} {filename}",
            "m"   : 20
            
        },
        {
            "name": "newt_methods",
            "cmd" : "NewtonMethods {method} {function_index} {x} {y} {max_iter} {filename}",
            "method" : 0,
            "function_index" : 2
        },
        {
            "name": "grad_desc",
            "cmd" : "GradientDescent {function_index} 1 {x} {y} {max_iter} {filename}"
        }
        ]

In [2]:
cmds[1]["cmd"].format(method=2,function_index=1,x=1,y=1,max_iter=20,filename="asdf")

'GradientDescent 1 1 1 1 20 asdf'

In [3]:
def create_cmd(cmd_entry,size,function_index=0,max_iter=1000000):
    spring = "wol" if function_index==0 else "wl"
    m = 0
    method = 0
    if "m" in cmd_entry:
        m = cmd_entry["m"]
    if "method" in cmd_entry:
        method = cmd_entry["method"]
    if "function_index" in cmd_entry and function_index == 1:
        function_index = cmd_entry["function_index"]
        spring = "wl_pHess"
    
    return cmd_entry["cmd"].format(function_index=function_index,
                                   x=size,
                                   y=size,
                                   max_iter=max_iter,
                                   filename=cmd_entry["name"]+"_"+spring+"_"+str(size)+"x"+str(size),
                                   m=m,
                                   method=method)

In [4]:
create_cmd(cmds[2],5,1)

'LBFGS 1 5 5 20 1000000 lbfgs_m20_wl_5x5'

# Different commands

❯ ./GaussNewton
Usage: input should be 'function index(0: f without length, 1: f with length), number of grid in x, number of grid in y, max iteration, filename', e.g. ./GaussNewton 1 20 20 10000 /usr/spring

❯ ./GradientDescent
Usage: input should be 'function index(0: f without length, 1: f with length),constrained spring scenario  (1 or 2 )number of grid in x, number of grid in y, max iteration, filename', e.g. ./GradientDescent 0 2 2 10000 /usr/spring

❯ ./LBFGS
Usage: input should be 'function index(0: f without length, 1: f with length), number of grid in x, number of grid in y, m, max iteration, filename', e.g. ./LBFGS 1 20 20 10 10000 /usr/spring

❯ ./NewtonMethods
Usage: input should be 'newton's method(0: standard newton, 1: projected hessian),function index(0: f without length, 1: f with length, 2: f with length with positive local hessian), number of grid in x, number of grid in y, max iteration, filename', e.g. ./NewtonMethods 0 0 2 2 10000 /usr/spring

In [5]:
import os
print(os.getcwd())

/Users/chris/Documents/GitHub/aopt/aopt-exercise7


In [6]:
os.chdir(path)


In [7]:
print(os.getcwd())

/Users/chris/Documents/GitHub/aopt/aopt-exercise7/cmake-build-debug/Build/bin


In [72]:
results = {
            "Spring without length": {},
            "Spring with length":    {}
}

### Regex match group to get numbers

group(1) = Total time<br>
group(2) = Total time evaluation<br>
group(3) = eval_f time<br>
group(4) = eval_grad time <br>
group(5) = eval_hessian time

In [71]:
import re
from subprocess import PIPE, Popen
size = 5
spring_type = "Spring without length"
for cmd_entry in cmds[:3]:
    results[spring_type][cmd_entry["name"]] = {
        "Command run" : [],
        "Total time" : [],
        "Total time evaluation" : [],
        "eval_f time" : [],
        "eval_grad time" : [],
        "eval_hessian time" : [],
        "Initial MassSpring system energy" : [],
        "Objective Function Value" : [],
    }
    
    #for size in dimensions:
    cmd = create_cmd(cmd_entry,size)
    results[spring_type][cmd_entry["name"]]["Command run"].append(cmd)
    p = Popen("./"+cmd, shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    system_energy_pattern = re.compile(r".*energy[ ]*is[ ](-?[\d.]+(?:e[+-]?\d+)?).*obj[ ]*=[ ]*(-?[\d.]+(?:e[+-]?\d+)?).*")
    for match in system_energy_pattern.finditer(str(stdout)):
        results[spring_type][cmd_entry["name"]]["Initial MassSpring system energy"].append((size,match.group(1)))
        results[spring_type][cmd_entry["name"]]["Objective Function Value"].append((size,match.group(2)))

    timing_pattern = re.compile(r".*total[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*total[ ]*time[ ]*evaluation[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_f[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_grad[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_hess[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*", re.MULTILINE | re.DOTALL)
    #print(cmd)
    #print(stderr)
    #print(stdout)
    for match in timing_pattern.finditer(str(stderr)):
        results[spring_type][cmd_entry["name"]]["Total time"].append((size,match.group(1)))
        results[spring_type][cmd_entry["name"]]["Total time evaluation"].append((size,match.group(2)))
        results[spring_type][cmd_entry["name"]]["eval_f time"].append((size,match.group(3)))
        results[spring_type][cmd_entry["name"]]["eval_grad time"].append((size,match.group(4)))
        results[spring_type][cmd_entry["name"]]["eval_hessian time"].append((size,match.group(5)))


    

In [88]:
import re
from subprocess import PIPE, Popen


for size in dimensions[:1]:
    spring_type = "Spring without length"
    
    cache = {}
    
    for cmd_entry in cmds:
        cache[cmd_entry["name"]] = {
            "Command run" : None,
            "Total time" : None,
            "Total time evaluation" : None,
            "eval_f time" : None,
            "eval_grad time" : None,
            "eval_hessian time" : None,
            "Initial MassSpring system energy" : None,
            "Objective Function Value" : None,
        }

        #for size in dimensions:
        cmd = create_cmd(cmd_entry,size)
        cache[cmd_entry["name"]]["Command run"]= cmd
        p = Popen("./"+cmd, shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = p.communicate()
        system_energy_pattern = re.compile(r".*energy[ ]*is[ ](-?[\d.]+(?:e[+-]?\d+)?).*obj[ ]*=[ ]*(-?[\d.]+(?:e[+-]?\d+)?).*")
        for match in system_energy_pattern.finditer(str(stdout)):
            cache[cmd_entry["name"]]["Initial MassSpring system energy"]=match.group(1)
            cache[cmd_entry["name"]]["Objective Function Value"]= match.group(2)

        timing_pattern = re.compile(r".*total[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*total[ ]*time[ ]*evaluation[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_f[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_grad[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_hess[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*", re.MULTILINE | re.DOTALL)
        #print(cmd)
        #print(stderr)
        #print(stdout)
        for match in timing_pattern.finditer(str(stderr)):
            cache[cmd_entry["name"]]["Total time"]= match.group(1)
            cache[cmd_entry["name"]]["Total time evaluation"]=match.group(2)
            cache[cmd_entry["name"]]["eval_f time"]=match.group(3)
            cache[cmd_entry["name"]]["eval_grad time"]=match.group(4)
            cache[cmd_entry["name"]]["eval_hessian time"]=match.group(5)
    results[spring_type][str(size)] = cache

    

In [89]:
df = pd.DataFrame(data=results["Spring without length"]["5"])
df

,gauss_newton,lbfgs_m20,newt_methods,grad_desc
Command run,GaussNewton 0 5 5 1000000 gauss_newton_wol_5x5,LBFGS 0 5 5 20 1000000 lbfgs_m20_wol_5x5,NewtonMethods 0 0 5 5 1000000 newt_methods_wol...,GradientDescent 0 1 5 5 1000000 grad_desc_wol_5x5
Total time,0.000656,0.000442,0.000263,0.264996
Total time evaluation,0,4e-06,8.4e-05,0.002473
eval_f time,0.00000,0.00000,0.00000,0.00047
eval_grad time,0.00000,0.00000,0.00000,0.00200
eval_hessian time,0.00000,0.00000,0.00008,0.00000
Initial MassSpring system energy,None,1.634e+07,1.634e+07,1.634e+07
Objective Function Value,None,1.634e+07,44.1344,44.1344
